# Rerun: Visualize everything fast!

Rerun is an SDK for visualizing multimodal data that changes over time. It’s used by engineers and researchers in fields like computer vision and robotics to verify, debug, and demo.

Rerun uses an open-core model. Everything in this repository will stay open source and free (both as in beer and as in freedom). In the future, Rerun will offer a commercial product that builds on top of the core free project.

The Rerun open source project targets the needs of individual developers. The commercial product targets the needs specific to teams that build and run computer vision and robotics products.

Rerun is an SDK and engine for visualizing and interacting with multimodal data streams.

Rerun is

- Simple to integrate and get started with
- Usable from Python, Rust, and C++
- Powerful, flexible, and extensible
- Built in Rust to be cross platform and fast
- Open source, dual licensed under MIT and Apache 2

Rerun is used by engineers and researchers in fields like computer vision and robotics to verify, debug, and demo.

## Use ReRun to

- Stream multimodal data from your code by logging it with the Rerun SDK
- Visualize and interact with live or recorded streams, whether local or remote

![](rerun.png)

## Installation:

To install Mito Sheet, execute the following cell in your JupyterLab environment:

In [1]:
%%capture
! pip install -U rerun-sdk

## Getting Started:

The simplest example is to open ReRun using the `spawn()` method:

![Empty Viewer](./img/rerun-empty.png)

In [1]:
import rerun as rr

rr.init()
rr.spawn()


thread 'ThreadId(1)' panicked at 'Failed to initialize any backend! Wayland status: XdgRuntimeDirNotSet X11 status: XOpenDisplayFailed'
winit-0.28.7/src/platform_impl/linux/mod.rs:757
stack backtrace:
   6: core::panicking::panic_fmt
             at core/src/panicking.rs:67:14
   7: eframe::native::run::create_event_loop
   8: eframe::native::run::with_event_loop
   9: eframe::run_native

Troubleshooting Rerun: https://www.rerun.io/docs/getting-started/troubleshooting
Report bugs: https://github.com/rerun-io/rerun/issues


But we can also view something simple in 3 space by generating some points and plotting them.

![Cube](./img/rerun-cube.png)

In [ ]:
import rerun as rr
import numpy as np

rr.init("rerun_example_my_data", spawn=True)

SIZE = 10

pos_grid = np.meshgrid(*[np.linspace(-10, 10, SIZE)]*3)
positions = np.vstack([d.reshape(-1) for d in pos_grid]).T

col_grid = np.meshgrid(*[np.linspace(0, 255, SIZE)]*3)
colors = np.vstack([c.reshape(-1) for c in col_grid]).astype(np.uint8).T

rr.log(
    "my_points",
    rr.Points3D(positions, colors=colors, radii=0.5)
)

But that's not that impressive. ReRun shines when we view multi-model time-series data:

In [2]:
from __future__ import annotations

import argparse
from math import tau

import numpy as np
import rerun as rr  # pip install rerun-sdk
from rerun.utilities import bounce_lerp, build_color_spiral

DESCRIPTION = """
# DNA
This is a minimal example that logs synthetic 3D data in the shape of a double helix. The underlying data is generated
using numpy and visualized using Rerun.

## How it was made
The full source code for this example is available
[on GitHub](https://github.com/rerun-io/rerun/blob/latest/examples/python/dna/main.py).

### Colored 3D points
The colored 3D points were added to the scene by logging the
[rr.Points3D archetype](https://www.rerun.io/docs/reference/types/archetypes/points3d) to the
[helix/structure/left](recording://helix/structure/left) and [helix/structure/right](recording://helix/structure/right)
entities.

### 3D line strips
The 3D line strips connecting the 3D point pairs are logged as an
[rr.LineStrips3D archetype](https://www.rerun.io/docs/reference/types/archetypes/line_strips3d) to the
[helix/structure/scaffolding entity](recording://helix/structure/scaffolding).

### Rotation
The whole structure is rotated over time by logging a
[rr.Transform3D archetype](https://www.rerun.io/docs/reference/types/archetypes/transform3d) to the
[helix/structure entity](recording://helix/structure.Transform3D) that changes over time. This transform determines the rotation of
the [structure entity](recording://helix/structure) relative to the [helix](recording://helix) entity. Since all other
entities are children of [helix/structure](recording://helix/structure) they will also rotate based on this transform.

You can visualize this rotation by selecting the two entities on the left-hand side and activating `Show transform` in
the Blueprint settings on the right-hand side. You will see one static frame (i.e., the frame of
[helix](recording://helix)) and the rotating frame (i.e., the frame of [structure](recording://helix/structure)).
""".strip()

rr.spawn()

rr.log("description", rr.TextDocument(DESCRIPTION, media_type=rr.MediaType.MARKDOWN), timeless=True)

rr.set_time_seconds("stable_time", 0)

NUM_POINTS = 100

# points and colors are both np.array((NUM_POINTS, 3))
points1, colors1 = build_color_spiral(NUM_POINTS)
points2, colors2 = build_color_spiral(NUM_POINTS, angular_offset=tau * 0.5)
rr.log("helix/structure/left", rr.Points3D(points1, colors=colors1, radii=0.08))
rr.log("helix/structure/right", rr.Points3D(points2, colors=colors2, radii=0.08))

rr.log("helix/structure/scaffolding", rr.LineStrips3D(np.stack((points1, points2), axis=1), colors=[128, 128, 128]))

time_offsets = np.random.rand(NUM_POINTS)
for i in range(400):
    time = i * 0.01
    rr.set_time_seconds("stable_time", time)

    times = np.repeat(time, NUM_POINTS) + time_offsets
    beads = [bounce_lerp(points1[n], points2[n], times[n]) for n in range(NUM_POINTS)]
    colors = [[int(bounce_lerp(80, 230, times[n] * 2))] for n in range(NUM_POINTS)]
    rr.log(
        "helix/structure/scaffolding/beads", rr.Points3D(beads, radii=0.06, colors=np.repeat(colors, 3, axis=-1))
    )

    rr.log(
        "helix/structure",
        rr.Transform3D(rotation=rr.RotationAxisAngle(axis=[0, 0, 1], radians=time / 4.0 * tau)),
    )


thread 'ThreadId(1)' panicked at 'Failed to initialize any backend! Wayland status: XdgRuntimeDirNotSet X11 status: XOpenDisplayFailed'
winit-0.28.7/src/platform_impl/linux/mod.rs:757
stack backtrace:
   6: core::panicking::panic_fmt
             at core/src/panicking.rs:67:14
   7: eframe::native::run::create_event_loop
   8: eframe::native::run::with_event_loop
   9: eframe::run_native

Troubleshooting Rerun: https://www.rerun.io/docs/getting-started/troubleshooting
Report bugs: https://github.com/rerun-io/rerun/issues
[2023-12-11T20:56:26Z WARN  re_sdk_comms::buffered_client] Failed to send message after 3 attempts: Failed to send to Rerun server at 127.0.0.1:9876: Broken pipe (os error 32)


[2023-12-11T20:56:29Z WARN  re_sdk_comms::buffered_client] Dropping messages because tcp client has timed out.
[2023-12-11T20:56:29Z WARN  re_sdk_comms::buffered_client] Dropping messages because tcp client has timed out.


![DNA](./img/rerun-dna.png)

![DNA](./img/rerun-dna.png)

In [4]:
#!/usr/bin/env python3
"""
Demonstrates how to log simple plots with the Rerun SDK.

"""
from __future__ import annotations

import argparse
import random
from math import cos, sin, tau

import numpy as np
import rerun as rr  # pip install rerun-sdk

DESCRIPTION = """
# Plots
This example shows various plot types that you can create using Rerun. Common usecases for such plots would be logging
losses or metrics over time, histograms, or general function plots.

## How it was made
The full source code for this example is available [on GitHub](https://github.com/rerun-io/rerun/blob/latest/examples/python/plots/main.py).

### Bar charts
The [bar chart](recording://bar_chart) is created by logging the [rr.BarChart archetype](https://www.rerun.io/docs/reference/types/archetypes/bar_chart).

### Time series
All other plots are created using the
[rr.TimeSeriesScalar archetype](https://www.rerun.io/docs/reference/types/archetypes/bar_chart)
with different settings. Each plot is created by logging scalars at different time steps (i.e., the x-axis).

For the [parabola](recording://curves/parabola) the radius and color is changed over time.

[sin](recording://trig/sin) and [cos](recording://trig/cos) are logged with the same parent entity (i.e.,
`trig/{cos,sin}`) which will put them in the same view by default.

For the [classification samples](recording://classification/samples) `rr.TimeSeriesScalar(..., scatter=True)` is used to
create separate points that do not connect over time. Note, that in the other plots the logged scalars are connected
over time by lines.
""".strip()

rr.spawn()

def clamp(n, smallest, largest):  # type: ignore[no-untyped-def]
    return max(smallest, min(n, largest))


def log_bar_chart() -> None:
    rr.set_time_sequence("frame_nr", 0)
    # Log a gauss bell as a bar chart
    mean = 0
    std = 1
    variance = np.square(std)
    x = np.arange(-5, 5, 0.1)
    y = np.exp(-np.square(x - mean) / 2 * variance) / (np.sqrt(2 * np.pi * variance))
    rr.log("bar_chart", rr.BarChart(y))


def log_parabola() -> None:
    # Log a parabola as a time series
    for t in range(0, 1000, 10):
        rr.set_time_sequence("frame_nr", t)

        f_of_t = (t * 0.01 - 5) ** 3 + 1
        radius = clamp(abs(f_of_t) * 0.1, 0.5, 10.0)
        color = [255, 255, 0]
        if f_of_t < -10.0:
            color = [255, 0, 0]
        elif f_of_t > 10.0:
            color = [0, 255, 0]

        rr.log(
            "curves/parabola",
            rr.TimeSeriesScalar(
                f_of_t,
                label="f(t) = (0.01t - 3)³ + 1",
                radius=radius,
                color=color,
            ),
        )


def log_trig() -> None:
    # Log a time series
    for t in range(0, int(tau * 2 * 100.0)):
        rr.set_time_sequence("frame_nr", t)

        sin_of_t = sin(float(t) / 100.0)
        rr.log("trig/sin", rr.TimeSeriesScalar(sin_of_t, label="sin(0.01t)", color=[255, 0, 0]))

        cos_of_t = cos(float(t) / 100.0)
        rr.log("trig/cos", rr.TimeSeriesScalar(cos_of_t, label="cos(0.01t)", color=[0, 255, 0]))


def log_classification() -> None:
    # Log a time series
    for t in range(0, 1000, 2):
        rr.set_time_sequence("frame_nr", t)

        f_of_t = (2 * 0.01 * t) + 2
        color = [255, 255, 0]
        rr.log("classification/line", rr.TimeSeriesScalar(f_of_t, color=color, radius=3.0))

        g_of_t = f_of_t + random.uniform(-5.0, 5.0)
        if g_of_t < f_of_t - 1.5:
            color = [255, 0, 0]
        elif g_of_t > f_of_t + 1.5:
            color = [0, 255, 0]
        else:
            color = [255, 255, 255]
        radius = abs(g_of_t - f_of_t)
        rr.log("classification/samples", rr.TimeSeriesScalar(g_of_t, color=color, scattered=True, radius=radius))



rr.log("description", rr.TextDocument(DESCRIPTION, media_type=rr.MediaType.MARKDOWN), timeless=True)
log_bar_chart()
log_parabola()
log_trig()
log_classification()


thread 'ThreadId(1)' panicked at 'Failed to initialize any backend! Wayland status: XdgRuntimeDirNotSet X11 status: XOpenDisplayFailed'
winit-0.28.7/src/platform_impl/linux/mod.rs:757
stack backtrace:
   6: core::panicking::panic_fmt
             at core/src/panicking.rs:67:14
   7: eframe::native::run::create_event_loop
   8: eframe::native::run::with_event_loop
   9: eframe::run_native

Troubleshooting Rerun: https://www.rerun.io/docs/getting-started/troubleshooting
Report bugs: https://github.com/rerun-io/rerun/issues
[2023-12-11T21:09:39Z WARN  re_sdk_comms::buffered_client] Failed to send message after 3 attempts: Failed to send to Rerun server at 127.0.0.1:9876: Broken pipe (os error 32)


![DNA](./img/rerun-multi-1.png)

The next example is more serious and will require us to run the code from a script.

In [9]:
! pip install segment_anything opencv_python torch torchvision tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 21.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.6 MB/s eta 0:00:00


In [10]:
! python scripts/rerun-segment_anything_model.py

Traceback (most recent call last):
  File "/workspaces/aaw/docs/en/1-Experiments/Notebooks/scripts/rerun-segment_anything_model.py", line 28, in <module>
    import cv2
  File "/usr/local/python/3.10.13/lib/python3.10/site-packages/cv2/__init__.py", line 181, in <module>
    bootstrap()
  File "/usr/local/python/3.10.13/lib/python3.10/site-packages/cv2/__init__.py", line 153, in bootstrap
    native_module = importlib.import_module("cv2")
  File "/usr/local/python/3.10.13/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
ImportError: libGL.so.1: cannot open shared object file: No such file or directory


See the official ReRun [Python Quickstart](https://www.rerun.io/docs/getting-started/python) for more information about ReRun and what it can do for you.